# Imports, load data

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import scipy
from scipy.special import gamma, gammaln, comb
from scipy.stats import beta as beta_dist
from abc import ABC, abstractmethod
import warnings

In [2]:
reader = pd.read_csv('full_cleaned_data.csv', chunksize=1)
for chunk in reader:
    column_names = chunk.columns.tolist()
    break
column_names.remove('CEPH ID')

In [3]:
cols = list(np.random.choice(column_names, size=1000, replace=False))+['CEPH ID']

data = pd.read_csv('full_cleaned_data.csv', usecols=cols).set_index('CEPH ID')

In [4]:
pops = pd.read_csv('HGDP/hgdp/HGDP-CEPH-ID_populations.csv').set_index('CEPH ID')
pops.head()

,population,Geographic origin,Region,Pop7Groups,Sex,All LCLs (H1063),Unrelated (1st and 2nd degree) (H951)
CEPH ID,,,,,,,
HGDP00001,Brahui,Pakistan,Asia,Central_South_Asia,M,yes,yes
HGDP00003,Brahui,Pakistan,Asia,Central_South_Asia,M,yes,yes
HGDP00005,Brahui,Pakistan,Asia,Central_South_Asia,M,yes,yes
HGDP00007,Brahui,Pakistan,Asia,Central_South_Asia,M,yes,yes
HGDP00009,Brahui,Pakistan,Asia,Central_South_Asia,M,yes,yes


# Combine

In [5]:
for c in ['population', 'Geographic origin', 'Region', 'Pop7Groups', 'Sex']:
    data[c] = pops[c]
    
# remove rows/samples with null values
data = data.drop(data[data.isnull().any(axis=1)].index.tolist())

data.head()

,rs10083789,rs10448293,rs10876889,rs11232065,rs11232180,rs1182425,rs12035887,rs12424570,rs12494110,rs1324790,...,rs9947051,rs9961688,rs999409,rs4568920,rs4128759,population,Geographic origin,Region,Pop7Groups,Sex
CEPH ID,,,,,,,,,,,,,,,,,,,,,
HGDP00448,2,2,0,2,2,2,2,2,2,2,...,0,0,2,2,1,Biaka_Pygmy,Central African Republic,Subsaharan Africa,Africa,M
HGDP00479,2,2,0,2,2,1,2,2,2,2,...,1,1,2,2,1,Biaka_Pygmy,Central African Republic,Subsaharan Africa,Africa,M
HGDP00985,2,1,1,2,2,1,2,0,2,2,...,1,2,2,2,2,Biaka_Pygmy,Central African Republic,Subsaharan Africa,Africa,M
HGDP01094,2,0,1,2,2,2,2,2,2,2,...,2,2,2,2,0,Biaka_Pygmy,Central African Republic,Subsaharan Africa,Africa,M
HGDP00982,2,2,1,2,2,0,2,2,2,2,...,2,2,2,2,2,Mbuti_Pygmy,Democratic Republic of Congo,Subsaharan Africa,Africa,M


# Model code

In [41]:
# New code
class BinomialModel(ABC):
    def __init__(self):
        pass
    
    def log_likelihood(self, data, theta, eps=1e-5):
        """
        Compute log P(D_k | theta) for some parameter settings theta.
        """
        S = theta.shape[0]
        N, M = data.shape
        
        C = np.log(comb(N=2, k=data).sum())
        
        # check for any theta that equal 0 or 1, add/subtract a small constant
        # (to avoid log(0) errors)
        idx = np.argwhere(np.isclose(theta, 0))
        theta[idx[:,0], idx[:,1]] += eps
        idx = np.argwhere(np.isclose(theta, 1))
        theta[idx[:,0], idx[:,1]] -= eps
        
        warnings.filterwarnings('error')
        try:
            T1 = np.dot(self.data, np.log(theta).T).sum(axis=0)
            T2 = np.dot((2 - self.data), np.log(1 - theta).T).sum(axis=0)
        except RuntimeWarning:
            print(theta)
            raise
        finally:
            warnings.filterwarnings('default')
        
        return C + T1 + T2
    
    @abstractmethod
    def log_marginal_likelihood(self, *args, **kwargs):
        pass
    
class BetaBinomial(BinomialModel):
    
    """
    When X ~ BetaBinomial(n, alpha, beta):
    P(X = k | n, alpha, beta) = G(n+1) / (G(k+1) G(n - k + 1))
                              * (G(k+alpha) G(n - k + beta)) / G(n + alpha + beta)
                              * G(alpha + beta) / (G(alpha) G(beta))
    where G is the gamma function, and 
    P(X = k | n, alpha, beta) = \int P(X = k | n, p) P(p | alpha, beta) dp
    
    See https://en.wikipedia.org/wiki/Beta-binomial_distribution
    """
    
    def __init__(self, alpha, beta, n=2):
        super(BetaBinomial, self).__init__()
        
        self.alpha = alpha
        self.beta = beta
        self.n = n
        
        # terms in the marginal log likelihood equation that are constant 
        # w.r.t. the data
        self.const = gammaln(n+1) - gammaln(n + alpha + beta) + gammaln(alpha + beta)
        self.const -= gammaln(alpha) + gammaln(beta)
        
    def log_marginal_likelihood(self, X):
        """
        TODO: doc
        """
        alpha, beta, n = self.alpha, self.beta, self.n
        
        # terms that are NOT constant w.r.t. X
        terms = - gammaln(X + 1) - gammaln(n - X + 1) + gammaln(X + alpha)
        terms = terms + gammaln(n - X + beta)
        
        return (terms + self.const).sum()

# Node/Leaf code

In [44]:
class Node(object):
    
    def __init__(self, data, alpha, model, left_child=None, right_child=None):
        self.data = data
        self.N = data.shape[0]
        self.alpha = alpha
        self.log_alpha = np.log(alpha)
        self.model = model
        
        self.log_pr_data_h1 = model.log_marginal_likelihood(data)
        
        self.left_child = left_child
        self.right_child = right_child
        if left_child is not None and right_child is not None:
            self.log_d = np.logaddexp(
                self.log_alpha + gammaln(self.N),
                self.left_child.log_d + self.right_child.log_d
            )
            
            self.log_pi = self.log_alpha + gammaln(self.N) - self.log_d
            
            log_1p = np.log(1. - np.exp(self.log_pi)) # = log(1 - pi_k)
            lpt_left = left_child.log_pr_data_tk
            lpt_right = right_child.log_pr_data_tk
            
            post_pr_h1 = self.log_pr_data_h1 + self.log_pi
            
            self.log_pr_data_tk = np.logaddexp(
                post_pr_h1, log_1p + lpt_left + lpt_right
            )
            
            self.log_rk = post_pr_h1 - self.log_pr_data_tk 
            
    @classmethod
    def merge(cls, left_child, right_child):
        assert left_child.alpha == right_child.alpha
        assert left_child.model == right_child.model
        
        data = np.vstack([left_child.data, right_child.data])
        return cls(
            data, left_child.alpha, left_child.model, left_child, right_child
        )

class Leaf(Node):
    
    def __init__(self, data, alpha, model):
        super(Leaf, self).__init__(data, alpha, model, None, None)
        
        self.log_d = self.log_alpha
        self.log_pi = 0.             # log(1)
        
        self.log_pr_data_tk = self.log_pr_data_h1
        self.log_rk = self.log_pr_data_h1
        

       

# Run

In [47]:
# a = 1.5
# b = 1.5

# xs = np.arange(0.01, 1., 0.01)
# ys = beta_dist.pdf(xs, a, b)
# ys /= np.sum(ys)

# f, ax = plt.subplots()
# ax.plot(xs, ys)
# ax.set_ylim([0., 0.1])
# plt.show()

model = BetaBinomial(alpha=1.5, beta=1.5)
crp_alpha = 1.

D1 = data.values[0, :50].astype(np.int8).reshape(1, -1)
n1 = Leaf(D1, crp_alpha, model)
D2 = data.values[1, :50].astype(np.int8).reshape(1, -1)
n2 = Leaf(D1, crp_alpha, model)

LL1 = n1.log_rk
LL2 = n2.log_rk

D3 = np.vstack([D1, D2])

n3 = Node(D3, crp_alpha, model, n1, n2)
np.exp(n3.log_rk)


0.6334310863707523